In [178]:
import pandas as pd
import numpy as np
import os
import glob
import plotly.express as px

experiment = "Experiments/1_anonymize_parties"
metrics_files = glob.glob(os.path.join(experiment, "metrics_*.csv"))
metrics_files

['Experiments/1_anonymize_parties/metrics_1a_parties_not_anonymous.csv',
 'Experiments/1_anonymize_parties/metrics_1b_parties_anonymous.csv']

In [176]:
metric_id = 0
df_plot = pd.DataFrame()
for file_name in metrics_files:
    df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
    metrics = list(set(df.columns.get_level_values(0)))
    experiment_run = "_".join(os.path.basename(file_name).split("_")[1:]).split(".")[0]
    df_plot[experiment_run] = df[metrics[metric_id]].mean(axis=0)


df = pd.DataFrame(df_plot).reset_index().rename(columns={"index": "party"})
df_melted = df.melt(id_vars=["party"], var_name="category", value_name="value")
print(df_melted.groupby("category")["value"].mean())
px.bar(
    df_melted,
    x="party",
    y="value",
    color="category",
    barmode="group",
    labels={"value": metrics[metric_id], "party": "Party"},
    title=f"{experiment}",
)

category
1a_parties_not_anonymous    0.836125
1b_parties_anonymous        0.750713
Name: value, dtype: float64


In [177]:
metric_id = 1
df_plot = pd.DataFrame()
for file_name in metrics_files:
    df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
    metrics = list(set(df.columns.get_level_values(0)))
    experiment_run = "_".join(os.path.basename(file_name).split("_")[1:]).split(".")[0]
    df_plot[experiment_run] = df[metrics[metric_id]].mean(axis=0)


df = pd.DataFrame(df_plot).reset_index().rename(columns={"index": "party"})
df_melted = df.melt(id_vars=["party"], var_name="category", value_name="value")
print(df_melted.groupby("category")["value"].mean())
px.bar(
    df_melted,
    x="party",
    y="value",
    color="category",
    barmode="group",
    labels={"value": metrics[metric_id], "party": "Party"},
    title=f"{experiment}",
)

category
1a_parties_not_anonymous    0.763600
1b_parties_anonymous        0.613239
Name: value, dtype: float64
